In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.autograd import Variable
from torchvision.transforms import ToTensor

from deeplib.datasets import load_cifar10, load_mnist, train_valid_loaders
from deeplib.net import CifarNet, CifarNetBatchNorm
from deeplib.training import train

## 1. Régularisation L1 et L2

In [ ]:
train_dataset, _ = load_cifar10(True, '/tmp')

### Implémentation manuelle
La régularisation L2 est communément appelée **weight decay**. Dans PyTorch, les optimiseur de torch.optim ont un paramètre `weight_decay` pour facilement utiliser cette régularisation. Par contre, on peut facilement implémenter manuellement la régularisation L2 si on l'interprète comme une pénalité sur la norme des poids (voir le [chapitre 7.1](http://www.deeplearningbook.org/contents/regularization.html)).

In [ ]:
def loss_init(parameters=[], reg_alpha=0):
    cross_entropy = nn.CrossEntropyLoss()
    
    def loss_function(output, targets):
        loss = cross_entropy(output,targets)
        
        for p in parameters:
            # TODO
            loss += reg_alpha * torch.norm(p)
        
        return loss
        
    return loss_function
    

net = CifarNet()
net.cuda()
optimizer = optim.SGD(net.parameters(), lr=0.1)
loss = loss_init(net.parameters(), 1e-3)

train(net, optimizer, train_dataset, n_epoch=5, batch_size=64, use_gpu=True, criterion=loss)

#### Exercice
- Implémenter la régularisation L2
- Implémenter la régularisation L1 manuellement (comme pour la L2)

#### En utilisant weight_decay

In [ ]:
batch_size = 64
lr = 0.2
n_epoch = 15
decay = 1e-4

net_without_l2 = CifarNet()
net_without_l2.cuda()
net_l2 = CifarNet()
net_l2.cuda()

optimizer_without_l2 = optim.SGD(net_without_l2.parameters(), lr=lr, weight_decay=0)
optimizer_l2 = optim.SGD(net_l2.parameters(), lr=lr, weight_decay=decay)

history_without_l2 = train(net_without_l2, optimizer_without_l2, train_dataset, n_epoch, batch_size, use_gpu=True)
history_l2 = train(net_l2, optimizer_l2, train_dataset, n_epoch, batch_size, use_gpu=True)

In [ ]:
history_without_l2.display_accuracy()

In [ ]:
history_l2.display_accuracy()

#### Question
- Quel est l'effet de la régularisation L2 sur l'entraînement du réseau, plus spécifiquement l'accuracy?

#### Exercice
- Utilisez un weight_decay trop grand. Qu'arrive-t-il? Pourquoi?
- Quel weight decay fonctionne le mieux pour vous? Un sondage sera fait lors du retour en classe à la fin du laboratoire.
- Dans la cellule suivante, analysez la distribution des poids appris par le réseau (variance, moyenne, min, max). Faites un histogramme des poids. Que remarquez-vous?

In [ ]:
def net_to_weight_array(net):
    weights = []
    for p in net.parameters():
        p_numpy = p.data.cpu().numpy()
        weights.append(p_numpy.reshape((-1))) # Reshape to 1D array
    return np.concatenate(weights)

        
# TODO
weights_without_l2 = net_to_weight_array(net_without_l2)
weights_l2 = net_to_weight_array(net_l2)

print(np.average(weights_without_l2))
print(np.average(weights_l2))
print(np.var(weights_without_l2))
print(np.var(weights_l2))
print(np.max(np.abs(weights_without_l2)))
print(np.max(np.abs(weights_l2)))

## 2. Early stopping

Commencez par entraîner un réseau un grand nombre d'epoch. L'historique d'entraînement nous servira de base pour les questions qui suivent.

In [ ]:
net = CifarNetBatchNorm()
net.cuda()
optimizer = optim.SGD(net.parameters(), lr=0.2, weight_decay=1e-4, nesterov=True, momentum=0.9)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[2,15,40], gamma=0.5)

history = train(net, optimizer, train_dataset, n_epoch=60, batch_size=64, use_gpu=True, scheduler=scheduler)

In [ ]:
history.display()

#### Questions
- En regardant les graphiques ci-dessus, quel est le meilleur moment pour arrêter l'entraînement? Pourquoi?
- Identifiez des problèmes pratiques potentiels lors de l'utilisation de l'early stopping.

#### Exercice
L'algorithme 7.1 du livre (voir http://www.deeplearningbook.org/contents/regularization.html) décrit le paramètre de patience `p`. Analysez l'effet du choix de `p` sur les données de l'entraînement précédent. Regardez, pour `p = 1,5,10,15`, quel modèle avec quelle précision en validation est choisi. Utilisez les `val_accuracy` de l'entraînement que vous venez d'exécuter pour vos tests (à la place d'entraîner le réseau).

In [ ]:
p = 1
val_accuracy = history.history['val_acc']

print(val_accuracy[:4])

## 3. Dropout
Cette section a pour but d'analyser l'effet du dropout dans un réseau fully connected. Nous ferons cette analyse en reprenant l'exercice du laboratoire 2.


In [ ]:
dataset, _ = load_mnist(True, '/tmp')

#### Exercice
Dans le réseau suivant, implémentez la fonction `forward()` en ajoutant du dropout si `self.use_dropout == True`. N'ajoutez **pas de softmax** car la fonction `deeplib.training.train()` utilise par défaut `CrossEntropyLoss`, ce qui le fait pour vous. Utilisez une probabilité de drop de `0.4`. Ne faites pas de dropout sur la dernière couche.

> **ATTENTION!** Vous devez bien fixer l'argument `training` de dropout. Vous pouvez savoir si modèle est en entraînement ou en évaluation avec self.training.

### Question
- Quelle est l'importance de l'argument `training` de la fonction de dropout?

In [ ]:
class MnistModel(torch.nn.Module):
    
    def __init__(self, n_layers, hidden_size=100, use_dropout=True):
        super().__init__()
        torch.manual_seed(12345)
        self.use_dropout = use_dropout
        self.hidden_size = hidden_size
        
        self.first_layer = nn.Linear(28*28,hidden_size)
        self.first_layer.weight.data.normal_(0.0, math.sqrt(2 / 28*28))
        self.first_layer.bias.data.fill_(0.0001)
        
        self.layers = []
        for i in range(n_layers - 1):
            layer = nn.Linear(hidden_size,hidden_size)
            layer.weight.data.normal_(0.0, math.sqrt(2 / hidden_size))
            layer.bias.data.fill_(0.0001)
            self.layers.append(layer)
            self.add_module('layer-%d' % i, layer)
            
        self.output_layer = nn.Linear(hidden_size,10)
        self.output_layer.weight.data.normal_(0.0, math.sqrt(2 / hidden_size))
        self.output_layer.bias.data.fill_(0.0001)

        self.nonzero_grad_stats = None
              

    def forward(self, x):
        x = x.view(-1, 28*28)
        out = F.relu(self.first_layer.forward(x))
        for i, l in enumerate(self.layers):
            out = F.relu(layer(out))
            if self.use_dropout and i < len(self.layers) - 1:
                out = F.dropout(out, 0.4, training=self.training)
        return self.output_layer.forward(out)

Entraînez un réseau avec dropout et un réseau sans dropout.

In [ ]:
net = MnistModel(2, use_dropout=False)
net_dropout = MnistModel(2, use_dropout=True)
net.cuda()
net_dropout.cuda()

optimizer = optim.SGD(net.parameters(), lr=0.005, nesterov=True, momentum=0.9)
optimizer_dropout = optim.SGD(net_dropout.parameters(), lr=0.005, nesterov=True, momentum=0.9)

history = train(net, optimizer, dataset, 20, batch_size=64)
history_dropout = train(net_dropout, optimizer_dropout, dataset, 20, batch_size=64)

In [ ]:
history.display()

In [ ]:
history_dropout.display()

#### Exercice
- Essayez plusieurs valeurs de dropout et observez les effets.
- Essayer d'avoir des valeurs différentes de probabilité de dropout pour chaque couche. Est-ce que cela améliore les résultats?